In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
logs_json = pd.read_json("/fastcampus-data/log-data/log_sampling.json")

In [3]:
df = pd.DataFrame(logs_json)

In [4]:
df.head()

,_metadata,anonymousId,channel,context,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,traits,type,userId,version,properties,category,name,event
0,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'campaign': {}, 'ip': '220.124.80.98', 'libra...",{},ajs-next-c1583afa7d4c5bf091c633d8232f9da3,2022-05-13T06:28:06.805Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:28:06.815Z,2022-05-13T06:28:06.805Z,2022-05-13 06:28:06.815,{'customerId': None},identify,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,None,None,None,None
1,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'ip': '220.124.80.98', 'library': {'name': 'a...",{},ajs-next-9b8d710113206ffe6539e3ecc587567e,2022-05-13T06:30:54.677Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:30:54.367Z,2022-05-13T06:30:54.677Z,2022-05-13 06:30:54.367,{'customerId': None},identify,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,None,None,None,None
2,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",623cef3e-6160-4ca8-ab57-f1046a946cb1,client,"{'ip': '220.124.80.98', 'library': {'name': 'a...",{},ajs-next-23c3daabb8984bada04b504404bbf197,2022-05-13T06:31:26.179Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:31:25.869Z,2022-05-13T06:31:26.179Z,2022-05-13 06:31:25.869,None,page,3471de99bb6b2a801b0b5946389150999832d5132f2c51...,2,"{'category': '', 'name': 'ArticlesArticle', 'p...",,ArticlesArticle,None
3,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",b50480bc-1b18-449a-a073-39275cea3d82,client,"{'campaign': {}, 'ip': '110.70.46.194', 'libra...",{},ajs-next-04e4e02a6a6444550572ec5e2612e8e5,2022-05-13T06:33:01.601Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:33:02.015Z,2022-05-13T06:33:01.601Z,2022-05-13 06:33:02.015,None,page,7f84cdf255bd825cbe17ec63c7cb0822ff8e5cc49201dc...,2,"{'path': '/', 'referrer': 'https://medistream....",None,None,None
4,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",497f3995-1387-4c97-85af-183bcefac9c9,client,"{'campaign': {}, 'ip': '104.28.102.64', 'libra...",{},ajs-next-4286d9e866e25b67b8451692a02f297a,2022-05-13T06:40:33.953Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T06:40:34.317Z,2022-05-13T06:40:33.953Z,2022-05-13 06:40:34.317,{'customerId': None},identify,d8d3d56a1089728ddc8e489d5c9144c71d526464d1c756...,2,None,None,None,None


In [5]:
# drop columns 전처리
def drop_columns(delete_columns:list,df:pd.DataFrame) -> pd.DataFrame:
    # 회의 때 얘기한 컬럼 전처리
    # None or 필요 없는 컬럼
    df = df.drop(columns=delete_columns)
    return df

def dict_to_column(columns:list,df:pd.DataFrame) -> pd.DataFrame:
    for col in columns:
        key_set = dict_to_set(col,df)
        df = set_to_column(col,key_set,df)
    return df

def dict_to_set(column:str,df:pd.DataFrame) -> set:
    key_set = set()
    for i in tqdm(df[column]):
        
        # column 내용이 dict일 경우
        if isinstance(i, dict):
            key_set |= set(i.keys())
        
        # column 내용이 None type 일 경우
        elif i == None:
            continue
        
        # column 내용이 [dict]로 감싸져있는 경우
        elif isinstance(i, list)&len(i)>0:
            if isinstance(i[0], str):
                continue
            key_set |= set(i[0].keys())
        


    return key_set

def set_to_column(column:str,key_set:set,df:pd.DataFrame) -> pd.DataFrame:
    for key in key_set:
        
        #중복인 경우 컬럼_중복컬럼으로 추가
        if key in df.columns:
            df[column+'_'+key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None
                                       else ( None \
                                       if len(x)==0 else ( x[0].get(key, None) \
                                       if isinstance(x[0],dict)  else \
                                           None)) \
                                      )
        #중복이 아닌 경우
        else:
            df[key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None
                                       else ( None \
                                       if len(x)==0 else ( x[0].get(key, None) \
                                       if isinstance(x[0],dict)  else \
                                           None)) \
                                      )

    df = df.drop(columns=[column])
    return df

# 컬럼안 key 값이 한 개일 경우 딕셔너리만 풉니다!
def key_to_element(element_columns:list,df:pd.DataFrame)->pd.DataFrame:
    for col in columns:
        key_set = dict_to_set(col, df)
        assert len(key_set)==1, f'{col}: key가 2개 이상이므로 dict_to_column 함수 이용하세요'
        for key in key_set:
            df[col] = df[col].apply(lambda x: x.get(key,None) if isinstance(x,dict) else None)
    return df

In [7]:
columns = ['_metadata','context','traits','properties']
df = dict_to_column(columns, logs_json)
delete_columns = ['properties_properties', 'properties_version','properties_type', \
                  'properties_category','properties_name']
df = drop_columns(delete_columns, df)

100%|██████████| 1019669/1019669 [00:01<00:00, 781241.47it/s] 


In [10]:
df = dict_to_column(['library','page'],df)

100%|██████████| 1019669/1019669 [00:01<00:00, 904154.88it/s]


In [30]:
df.columns

Index(['anonymousId', 'channel', 'integrations', 'messageId',
       'originalTimestamp', 'projectId', 'receivedAt', 'sentAt', 'timestamp',
       'type', 'userId', 'version', 'category', 'name', 'event', 'bundled',
       'bundledIds', 'unbundled', 'locale', 'userAgent', 'campaign', 'ip',
       'customerId', 'title', 'search', 'properties_userId', 'url', 'path',
       'referrer', 'library_version', 'library_name', 'page_title',
       'page_search', 'page_url', 'page_path', 'page_referrer'],
      dtype='object')

In [38]:
df_customerId = df[~df['customerId'].isna()]

# product sorting

In [23]:
# product 페이지로 넘어간 log
product_show = df[df['path'].apply(lambda x : x.startswith('/product') if isinstance(x, str) else False)]

In [34]:
# product 를 본사람 중에 customerId가 존재하는 경우가 없다??? 흠...
product_show[~product_show['customerId'].isna()]

,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,category,name,event,bundled,bundledIds,unbundled,locale,userAgent,campaign,ip,customerId,title,search,properties_userId,url,path,referrer,library_version,library_name,page_title,page_search,page_url,page_path,page_referrer


In [42]:
df_customerId[df_customerId['page_url'].str.contains('product')]

,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,category,name,event,bundled,bundledIds,unbundled,locale,userAgent,campaign,ip,customerId,title,search,properties_userId,url,path,referrer,library_version,library_name,page_title,page_search,page_url,page_path,page_referrer
121,69a2af0f-aae2-4162-a9c4-25b720c59465,client,{},ajs-next-85a92fd309438ce6e945578051e6afa3,2022-05-13T08:35:03.823Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T08:35:03.945Z,2022-05-13T08:35:03.823Z,2022-05-13 08:35:03.945,identify,d59bbdc0adb5948756e74b8ad0854a691f6355a7c071cb...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like...,{},104.28.84.66,5e4fac8d4267e105dfff2082,None,None,None,None,None,None,next-1.36.0,analytics.js,메디스트림 | Medistream,?keyword=%EC%8B%A0%EC%A1%B8&type=product,https://medistream.co.kr?keyword=%EC%8B%A0%EC%...,/,https://medistream.co.kr/
2404,894ecdd5-393d-4688-8175-1220d8a78a44,client,{},ajs-next-ec35034d2df2fa556f0c3cd63b0d63af,2022-05-13T07:09:34.992Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T07:09:35.151Z,2022-05-13T07:09:34.992Z,2022-05-13 07:09:35.151,identify,7c62963828569149d74ffafc4d6fd2e9eb701acee7a850...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like...,{},104.28.84.65,5d70c01119efa30eb2914252,None,None,None,None,None,None,next-1.36.0,analytics.js,메디스트림 | Medistream,?keyword=%ED%95%9C%EC%9D%98%EC%82%AC%EB%A5%BC%...,https://medistream.co.kr?keyword=%ED%95%9C%EC%...,/,https://nid.naver.com/
3019,69a2af0f-aae2-4162-a9c4-25b720c59465,client,{},ajs-next-65c0e0e85590fd0fa56e55ce35fbf58f,2022-05-13T08:34:59.912Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T08:35:00.032Z,2022-05-13T08:34:59.912Z,2022-05-13 08:35:00.032,identify,d59bbdc0adb5948756e74b8ad0854a691f6355a7c071cb...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like...,{},104.28.84.66,5e4fac8d4267e105dfff2082,None,None,None,None,None,None,next-1.36.0,analytics.js,메디스트림 | Medistream,?type=product&placeholder=%EB%A9%94%EB%94%94%E...,https://medistream.co.kr?type=product&placehol...,/,https://medistream.co.kr/
3076,be87a96b-f0d6-4b8d-b67e-4c47be5fe5c0,client,{},ajs-next-eba7038f513693833d8b125abf9a8104,2022-05-13T11:10:30.351Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T11:06:32.873Z,2022-05-13T11:10:30.351Z,2022-05-13 11:06:32.873,identify,960f5453c61ca6ba73e8d20b19a90429b51bb82d3fb98f...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},119.203.224.86,5d69bf3919efa30eb2913dd8,None,None,None,None,None,None,next-1.36.0,analytics.js,메디스트림 | Medistream,?keyword=%EA%B2%BD%EC%98%A5%EA%B3%A0&type=product,https://medistream.co.kr?keyword=%EA%B2%BD%EC%...,/,https://nid.naver.com/
4805,1a6e3df6-202f-4ad4-84eb-80cee0e27f44,client,{},ajs-next-03f9aafad9870c62aac7c82438fbd0f3,2022-05-13T04:33:30.437Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T04:33:30.583Z,2022-05-13T04:33:30.437Z,2022-05-13 04:33:30.583,identify,f9458b512d12e5b50026797e83520b21bc796fb5afadba...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like...,{},39.7.230.167,5d6ba13119efa30eb2913e1e,None,None,None,None,None,None,next-1.36.0,analytics.js,메디스트림 | Medistream,?keyword=%ED%98%88%EC%95%A1%EA%B2%80%EC%82%AC&...,https://medistream.co.kr?keyword=%ED%98%88%EC%...,/,https://medistream.co.kr/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018732,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-67a0f9543e68ef03dbaed2e3c3212f06,2022-08-24T10:37:16.459Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T10:37:17.439Z,2022-08-24T10:37:16.530Z,2022-08-24 10:37:17.368,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,None,59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.41.0,analytics.js,메디스트림 |

# User 추적

## 5df202e44267e105dffef7d3 유저 추적

In [44]:
user_1 = df_customerId[df_customerId['customerId'] == '5df202e44267e105dffef7d3']

In [51]:
# index 1014521 8월 24일 갑자기 shopping으로 들어와서 쇼핑을 시작함..
user_1.iloc[-50:,:]

,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,category,name,event,bundled,bundledIds,unbundled,locale,userAgent,campaign,ip,customerId,title,search,properties_userId,url,path,referrer,library_version,library_name,page_title,page_search,page_url,page_path,page_referrer
928812,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-033f9d4adf054bec5c61f352160e2e67,2022-05-21T00:08:34.236Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T00:08:35.476Z,2022-05-21T00:08:34.236Z,2022-05-21 00:08:35.476,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?sort=three_months%3Adesc,https://medistream.co.kr?sort=three_months%3Adesc,/,https://nid.naver.com/
928994,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-01e76c7a181fc289296647488e0f94ab,2022-05-21T03:04:50.674Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T03:04:51.951Z,2022-05-21T03:04:50.674Z,2022-05-21 03:04:51.951,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?placeholder=%EB%A9%94%EB%94%94%EC%8A%A4%ED%8A...,https://medistream.co.kr?placeholder=%EB%A9%94...,/,https://nid.naver.com/
929165,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-07f6a99d0236a0a468c25ba95c6c278e,2022-05-21T03:04:24.765Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T03:04:26.352Z,2022-05-21T03:04:24.765Z,2022-05-21 03:04:26.352,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?keyword=%EC%8B%9D%EC%97%BC%EC%88%98&sort=thre...,https://medistream.co.kr?keyword=%EC%8B%9D%EC%...,/,https://nid.naver.com/
929277,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-25ce279fb7318eee3ee43a8ca515444a,2022-05-21T03:04:32.371Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T03:04:33.647Z,2022-05-21T03:04:32.371Z,2022-05-21 03:04:33.647,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?keyword=ns&sort=three_months%3Adesc,https://medistream.co.kr?keyword=ns&sort=three...,/,https://nid.naver.com/
929560,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-2b00fc159e338bad73429cadbd331aab,2022-05-21T03:04:32.485Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T03:04:33.761Z,2022-05-21T03:04:32.485Z,2022-05-21 03:04:33.761,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?sort=three_months%3Adesc,https://medistream.co.kr?sort=three_months%3Adesc,/,https://nid.naver.com/
929708,4925dade-2014-4580-9b95-5e3d77b6767d,client,{},ajs-next-7f8490fc82b04d6bd339b815e8eabf6b,2022-05-21T03:04:42.454Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-21T03:04:43.731Z,2022-05-21T03:04:42.454Z,2022-05-21 03:04:43.731,identify,de1b133185304a11f662479282bc8a78a28f096e830284...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,{},59.22.91.9,5df202e44267e105dffef7d3,None,None,None,None,None,None,next-1.37.0,analytics.js,(1) 메디스트림 | Medistream,?placeholder=%EB%A9%94%EB%94%94%EC%8A%A4%ED%8A...,https://medistream.co.kr?placeholder=%EB%A9%94...,/,https://nid.naver.com/
929967,4925dade-2014-4580-9b95-5e3d77b6767d,client,{

In [53]:
user_1.loc[991855]['page_url']

'https://medistream.co.kr?redirect=aHR0cHM6Ly93aWtpLm1lZGlzdHJlYW0uY28ua3IvaW5kZXgucGhwP3RpdGxlPSVFRCU4QSVCOSVFQyU4OCU5ODpPQXV0aDJDbGllbnQvcmVkaXJlY3QmcmV0dXJudG89JUVCJThDJTgwJUVCJUFDJUI4&code=VTJGc2RHVmtYMS81NktCdGNrYjFDTmszMnFMaGNxWXBZaTEwc2NSa2RuejNlMGJKWk1PdzVCcllLRW5Nd3NCOFFQeTd0RWFLY05yeWdISGk1RXYyR3c9PQ%3D%3D'

## 5ef1cb9be510ee503167d359

In [54]:
user_2 = df_customerId[df_customerId['customerId'] == '5ef1cb9be510ee503167d359']

In [55]:
user_2.iloc[-50:,:]

,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,category,name,event,bundled,bundledIds,unbundled,locale,userAgent,campaign,ip,customerId,title,search,properties_userId,url,path,referrer,library_version,library_name,page_title,page_search,page_url,page_path,page_referrer
164710,24296a8a-a8d3-400f-aaa8-4a06251f7146,client,{},ajs-next-d4c424d8be0f38d43118f3f8e3849bed,2022-06-25T11:47:10.251Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-06-25T11:47:10.588Z,2022-06-25T11:47:10.251Z,2022-06-25 11:47:10.588,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,None,114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.38.0,analytics.js,메디스트림 | Medistream,,https://medistream.co.kr,/,https://nid.naver.com/
210312,24296a8a-a8d3-400f-aaa8-4a06251f7146,client,{},ajs-next-b9d1efa04f43b579397804e448df41ca,2022-07-02T05:20:01.499Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-07-02T05:20:02.273Z,2022-07-02T05:20:01.503Z,2022-07-02 05:20:02.269,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,{},39.7.28.164,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.39.1,analytics.js,메디스트림 | Medistream,?redirect=L2FydGljbGVzLzUwNjAyP3V0bV9zb3VyY2U9...,https://medistream.co.kr?redirect=L2FydGljbGVz...,/,https://medistream.co.kr/
252269,ca072212-bd7a-4cba-a488-6808aa59484e,client,{},ajs-next-219099575aaa25d4a756bed483f926cf,2022-07-09T16:25:45.374Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-07-09T16:25:46.405Z,2022-07-09T16:25:45.434Z,2022-07-09 16:25:46.345,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,None,114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.39.1,analytics.js,메디스트림 | Medistream,,https://medistream.co.kr,/,https://nid.naver.com/
255479,ca072212-bd7a-4cba-a488-6808aa59484e,client,{},ajs-next-ce2c089cc7d848c1975df4dacfe4b12e,2022-07-09T16:24:20.835Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-07-09T16:24:21.675Z,2022-07-09T16:24:20.844Z,2022-07-09 16:24:21.666,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,{},114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.39.1,analytics.js,메디스트림 | Medistream,?redirect=L2FydGljbGVzLzUwNzgzP3V0bV9zb3VyY2U9...,https://medistream.co.kr?redirect=L2FydGljbGVz...,/,https://nid.naver.com/
306589,f32634fb-0d99-42ff-b4f7-3366817dc5b2,client,{},ajs-next-aed80c7762a80b07bfbec88984ad3868,2022-08-29T14:42:01.259Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-29T14:42:01.705Z,2022-08-29T14:42:01.373Z,2022-08-29 14:42:01.591,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,{},114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.41.0,analytics.js,메디스트림 | Medistream,?redirect=L21hcmtldD91dG1fc291cmNlPWtha2FvdGFs...,https://medistream.co.kr/auth?redirect=L21hcmt...,/auth,https://medistream.co.kr/
306741,f32634fb-0d99-42ff-b4f7-3366817dc5b2,client,{},ajs-next-9d4f8eb4c61a6988236cf9ead20b391d,2022-08-29T14:42:13.030Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-29T14:42:13.127Z,2022-08-29T14:42:13.038Z,2022-08-29 14:42:13.119,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,{},114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.41.0,analytics.js,메디스트림 | Medistream,?sort=date_created%3Adesc,https://medistream.co.kr/market/categories/62f...,/market/categories/62f609c39c24785b4c5e0be7,https://medistream.co.kr/
307887,f32634fb-0d99-42ff-b4f7-3366817dc5b2,client,{},ajs-next-dfcc1dde4952b3d29940b7479d7041aa,2022-0

In [57]:
# 이 글을 보고 product 란으로 넘어감
user_2.loc[1012919]['page_url']

'https://medistream.co.kr/auth?redirect=L21hcmtldD91dG1fc291cmNlPWtha2FvdGFsayZ1dG1fbWVkaXVtPWdvbmdqaSZ1dG1fY2FtcGFpZ249JUVCJThCJUE4JUVEJTg2JUExJUVCJUIwJUE5JTIwJUVBJUIzJUI1JUVDJUE3JTgwJnV0bV90ZXJtPSU1QiVFQiU4QiVBOCVFRCU4NiVBMSVFQiVCMCVBOSVFQSVCMyVCNSVFQyVBNyU4MCU1RCUyMCVFQiVBNyU4OCVFQyVCQyU5MyZ1dG1fY29udGVudD1kYW50YWxr&code=VTJGc2RHVmtYMThaeEFuQ2M1K3VjODZ6cEg1dzRRaUhCVjVad2czcXR4c2ZnSWpOUm1wSXBPRWNBUThTWXRHOGJaSnorU1BOTytBWnpDSmV3eHpPYlE9PQ%3D%3D'

In [60]:
# 혹시 이 글을 본 사람이 더 있나.. 없다.
df_customerId[df_customerId['page_url'] == 'https://medistream.co.kr/auth?redirect=L21hcmtldD91dG1fc291cmNlPWtha2FvdGFsayZ1dG1fbWVkaXVtPWdvbmdqaSZ1dG1fY2FtcGFpZ249JUVCJThCJUE4JUVEJTg2JUExJUVCJUIwJUE5JTIwJUVBJUIzJUI1JUVDJUE3JTgwJnV0bV90ZXJtPSU1QiVFQiU4QiVBOCVFRCU4NiVBMSVFQiVCMCVBOSVFQSVCMyVCNSVFQyVBNyU4MCU1RCUyMCVFQiVBNyU4OCVFQyVCQyU5MyZ1dG1fY29udGVudD1kYW50YWxr&code=VTJGc2RHVmtYMThaeEFuQ2M1K3VjODZ6cEg1dzRRaUhCVjVad2czcXR4c2ZnSWpOUm1wSXBPRWNBUThTWXRHOGJaSnorU1BOTytBWnpDSmV3eHpPYlE9PQ%3D%3D']

,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,category,name,event,bundled,bundledIds,unbundled,locale,userAgent,campaign,ip,customerId,title,search,properties_userId,url,path,referrer,library_version,library_name,page_title,page_search,page_url,page_path,page_referrer
1012919,f32634fb-0d99-42ff-b4f7-3366817dc5b2,client,{},ajs-next-8d1bd9673a31dd2a8e874e50bd4318b8,2022-08-24T14:20:05.342Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T14:20:05.885Z,2022-08-24T14:20:05.437Z,2022-08-24 14:20:05.790,identify,00b602eb919b9f41dc72e878f3e8476e669506b91c0c30...,2,None,None,None,[Segment.io],[],[],ko-KR,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,{},114.202.181.109,5ef1cb9be510ee503167d359,None,None,None,None,None,None,next-1.41.0,analytics.js,메디스트림 | Medistream,?redirect=L21hcmtldD91dG1fc291cmNlPWtha2FvdGFs...,https://medistream.co.kr/auth?redirect=L21hcmt...,/auth,https://nid.naver.com/


## user 5ea1aa7abfe4260944786332 추적

In [141]:
logs_json[logs_json['traits'].apply(lambda x:x['customerId'] if type(x)==dict else None)=='5ea1aa7abfe4260944786332'].iloc[-20:,:].tail()

,_metadata,anonymousId,channel,context,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,traits,type,userId,version,properties,category,name,event,unbundled,bundledIds,bundled,_metadata_unbundled,_metadata_bundledIds,_metadata_bundled,customerId,traits_customerId
1017543,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",27c6f174-e9d1-4f19-a412-bf5a101e3369,client,"{'ip': '211.106.115.80', 'library': {'name': '...",{},ajs-next-04e63a66f4d8decd4dee310514f5b7e7,2022-08-24T10:47:10.758Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T10:47:10.999Z,2022-08-24T10:47:10.782Z,2022-08-24 10:47:10.975,{'customerId': '5ea1aa7abfe4260944786332'},identify,a2cf6a4a38161b7bb6ee44fc8353e26944763708acad35...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5ea1aa7abfe4260944786332,5ea1aa7abfe4260944786332
1018486,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",27c6f174-e9d1-4f19-a412-bf5a101e3369,client,"{'ip': '211.106.115.80', 'library': {'name': '...",{},ajs-next-6f9df0def0a4e6acec1a37ce5925f7ea,2022-08-24T09:40:00.315Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T09:40:00.365Z,2022-08-24T09:40:00.323Z,2022-08-24 09:40:00.357,{'customerId': '5ea1aa7abfe4260944786332'},identify,a2cf6a4a38161b7bb6ee44fc8353e26944763708acad35...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5ea1aa7abfe4260944786332,5ea1aa7abfe4260944786332
1019020,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",27c6f174-e9d1-4f19-a412-bf5a101e3369,client,"{'campaign': {}, 'ip': '211.106.115.80', 'libr...",{},ajs-next-3c5776033cfcd1abd95a84969226a15e,2022-08-24T09:40:00.437Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T09:40:00.568Z,2022-08-24T09:40:00.460Z,2022-08-24 09:40:00.545,{'customerId': '5ea1aa7abfe4260944786332'},identify,a2cf6a4a38161b7bb6ee44fc8353e26944763708acad35...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5ea1aa7abfe4260944786332,5ea1aa7abfe4260944786332
1019098,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",27c6f174-e9d1-4f19-a412-bf5a101e3369,client,"{'ip': '211.106.115.80', 'library': {'name': '...",{},ajs-next-33d1d415b1cccbe474af8746f8b0a17a,2022-08-24T09:39:58.969Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T09:39:59.041Z,2022-08-24T09:39:58.976Z,2022-08-24 09:39:59.034,{'customerId': '5ea1aa7abfe4260944786332'},identify,a2cf6a4a38161b7bb6ee44fc8353e26944763708acad35...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5ea1aa7abfe4260944786332,5ea1aa7abfe4260944786332
1019627,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",27c6f174-e9d1-4f19-a412-bf5a101e3369,client,"{'ip': '211.106.115.80', 'library': {'name': '...",{},ajs-next-ac45579d4a135fe9b2cf513895cb8aad,2022-08-24T09:40:21.480Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T09:40:21.574Z,2022-08-24T09:40:21.519Z,2022-08-24 09:40:21.535,{'customerId': '5ea1aa7abfe4260944786332'},identify,a2cf6a4a38161b7bb6ee44fc8353e26944763708acad35...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5ea1aa7abfe4260944786332,5ea1aa7abfe4260944786332


In [56]:
# drop columns 전처리
def drop_columns(delete_columns:list,df:pd.DataFrame) -> pd.DataFrame:
    # 회의 때 얘기한 컬럼 전처리
    # None or 필요 없는 컬럼
    df = df.drop(columns=delete_columns)
    return df

def dict_to_column(columns:list,df:pd.DataFrame) -> pd.DataFrame:
    for col in columns:
        key_set = dict_to_set(col,df)
        df = set_to_column(col,key_set,df)
    return df

def dict_to_set(column:str,df:pd.DataFrame) -> set:
    key_set = set()
    for i in tqdm(df[column]):
        
        # column 내용이 dict일 경우
        if isinstance(i, dict):
            key_set |= set(i.keys())
        
        # column 내용이 None type 일 경우
        elif i == None:
            continue
        
        # column 내용이 [dict]로 감싸져있는 경우
        elif isinstance(i, list)&len(i)>0:
            if isinstance(i[0], str):
                continue
            key_set |= set(i[0].keys())
        


    return key_set

def set_to_column(column:str,key_set:set,df:pd.DataFrame) -> pd.DataFrame:
    for key in key_set:
        
        #중복인 경우 컬럼_중복컬럼으로 추가
        if key in df.columns:
            df[column+'_'+key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None
                                       else ( None \
                                       if len(x)==0 else ( x[0].get(key, None) \
                                       if isinstance(x[0],dict)  else \
                                           None)) \
                                      )
        #중복이 아닌 경우
        else:
            df[key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None
                                       else ( None \
                                       if len(x)==0 else ( x[0].get(key, None) \
                                       if isinstance(x[0],dict)  else \
                                           None)) \
                                      )

    df = df.drop(columns=[column])
    return df

# 컬럼안 key 값이 한 개일 경우 딕셔너리만 풉니다!
def key_to_element(element_columns:list,df:pd.DataFrame)->pd.DataFrame:
    for col in columns:
        key_set = dict_to_set(col, df)
        assert len(key_set)==1, f'{col}: key가 2개 이상이므로 dict_to_column 함수 이용하세요'
        for key in key_set:
            df[col] = df[col].apply(lambda x: x.get(key,None) if isinstance(x,dict) else None)
    return df

## 임상해부학(60236e092c593a001ace9ffa) 추적 

In [70]:
orders_json = pd.read_json("/fastcampus-data/orders/orders.json")

In [71]:
orders_df = pd.DataFrame(orders_json)
orders_items = dict_to_column(['items'],orders_df)

100%|██████████| 206689/206689 [00:00<00:00, 263494.13it/s]


In [72]:
orders_items = dict_to_column(['customer_id','product_id'],orders_items)

100%|██████████| 206689/206689 [00:00<00:00, 801276.71it/s]


In [83]:
orders_items[orders_items['product_id_$oid']== '60236e092c593a001ace9ffa']['$oid'][-20:]

186472    5d9eec7f0dabe405b156e269
186477    5d9eec7f0dabe405b156e269
186524    5dd1f6d82bb59605ca3d00b2
186943    5d835e463f0e6805c4706842
186981    607860ab0ccc56001bc7b697
191058    62980e059d9388002407182f
191069    62980e059d9388002407182f
191078    62980e059d9388002407182f
191079    62980e059d9388002407182f
191254    5e89af1dbfe4260944784bc6
193571    62980e059d9388002407182f
197579    60617cd80ccc56001bc7a025
197586    5ed8c840e510ee503167c752
197782    602cbdf6527223001a2095d6
197811    5f72c57448060c52caf15b98
198042    62980e059d9388002407182f
198227    5e01d8a24267e105dffeff76
199871    5e60c45ebfe4260944782f20
202897    5e576bdebfe42609447829bc
206218    5da831d80dabe405b156ec10
Name: $oid, dtype: object

In [123]:
orders_items[orders_items['$oid'] =='5f7fdd7348060c52caf165c6']

,_id,date_created,date_placed,date_updated,date_closed,date_paid,date_cancelled,number,shipping_status,discounts,tax_rate,shipping_tax,shipping_discount,shipping_price,item_tax_included,shipping_tax_included,closed,cancelled,delivered,paid,hold,draft,referrer_url,landing_url,channel,note,comments,coupon,tracking_number,status_id,payment_method_id,shipping_method_id,tags,price,itemIds,category_ids,price_total,price_detail,weight,variant_name,product_image,discount_total,tax_class,variant_id,sku,quantity,custom_note,tax_total,name,origin_items,id,custom_price,category_id,$oid,product_id_$oid
34457,{'$oid': '60056021a6432d001a69b67a'},{'$date': '2021-01-18T10:17:05.928Z'},{'$date': '2021-01-18T10:18:48Z'},{'$date': '2021-01-21T14:29:05.166Z'},None,{'$date': '2021-01-19T02:07:32.589Z'},None,60941,,[],0,0,0,0,True,True,False,False,False,True,False,False,,,,,,,,{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[],43000.0,None,None,43000.0,None,0.0,,"[{'id': {'$oid': '5de0b9c82bb59605ca3d0b4a'}, ...",0.0,면세,None,BO-MB-MGGS-DJGBJ-OTH,1.0,{},0.0,등중갑 방제학강의,None,{'$oid': '60056022a6432d001a69b67b'},{},None,5f7fdd7348060c52caf165c6,5ddf84182bb59605ca3d0afe
34533,{'$oid': '60065247a6432d001a69b77e'},{'$date': '2021-01-19T03:30:15.290Z'},None,None,None,None,None,61121,,[],0,0,0,0,True,True,False,False,False,False,False,True,,,,,,,,{'$oid': '5d1d485b4b25b80d9fb1f372'},None,None,[],NaN,None,None,NaN,None,NaN,None,None,NaN,None,None,None,NaN,None,NaN,None,None,None,None,None,5f7fdd7348060c52caf165c6,None
34577,{'$oid': '60069f572c593a001ace82e4'},{'$date': '2021-01-19T08:59:03.903Z'},{'$date': '2021-01-19T23:34:53Z'},{'$date': '2021-01-20T02:31:13.777Z'},None,{'$date': '2021-01-19T23:34:53.222Z'},None,61221,,"[{'id': {'$oid': '60076c3d2c593a001ace833a'}, ...",0,0,0,0,True,True,False,False,False,True,False,False,,,,,,,,{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d58dfb04e77525ec5ca11dd'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[],48450.0,None,None,48450.0,None,0.0,,"[{'id': {'$oid': '5f1550db221c8b5f77c8f361'}, ...",0.0,면세,None,BO-MB-GAON-ZSWKI-OTH,1.0,{},0.0,임상 한의사를 위한 주성완 원장 강의록,None,{'$oid': '60069f582c593a001ace82e5'},{},None,5f7fdd7348060c52caf165c6,5f1550c2221c8b5f77c8f35d
36010,{'$oid': '6019cbd12c593a001ace95dc'},{'$date': '2021-02-02T22:01:53.305Z'},{'$date': '2021-02-11T21:04:15Z'},{'$date': '2021-02-15T02:32:06.729Z'},None,{'$date': '2021-02-11T21:04:15.474Z'},None,64677,,"[{'id': {'$oid': '60259bb3527223001a208e23'}, ...",0,0,0,0,True,True,False,False,False,True,False,False,,,,,,,,{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d58dfb04e77525ec5ca11dd'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[],112500.0,None,None,112500.0,None,0.0,,"[{'id': {'$oid': '5d78568619efa30eb29143d9'}, ...",0.0,면세,None,BO-MB-KOON-TJSNB-OTH,1.0,{},0.0,개원의를 위한 통증사냥법,None,{'$oid': '602598ba2c593a001acea637'},{},None,5f7fdd7348060c52caf165c6,5d78505019efa30eb29143d2
37101,{'$oid': '6024a6b6527223001a208c5d'},{'$date': '2021-02-11T03:38:30.245Z'},{'$date': '2021-02-11T03:39:26Z'},{'$date': '2021-02-15T09:50:00.519Z'},None,{'$date': '2021-02-11T03:39:26.993Z'},None,67193,,[],0,0,0,0,True,True,False,False,False,True,False,False,,,,- [정현섭]님이 [21년2월15일 18시49분]에 수동 [배송주문]처리,,,,{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d58dfb04e77525ec5ca11dd'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[],100.0,None,None,100.0,None,0.0,강의 목록: 임상해부학 특강: 골학,"[{'id': {'$oid': '602384652c593a001acea01d'}, ...",0.0,과세,{'$oid': '60236f102c593a001acea003'},CT-LT-MEDI-CLANA-OTH,1.0,{},0.0,임상해부학 특강,None,{'$oid': '6024a6b6527223001a208c5e'},{},None,5f7fdd7348060c52caf165c6,60236e092c593a001ace9ffa
39669,{'$oid': '604739630ccc56001bc7847c'},{'$date': '2021-03-09T09:01:23.643Z'},{'$date': '2021-03-21T08:06:55Z'},{'$date': '2021-03-22T02:32:13.080Z'},None,{'$date': '2021-03-21T08:06:55.258Z'},None,73249,,"[{'id': {'$oid': '6056fe89c1908b001a63b8ea'}, ...",0,0,0,0,True,False,False,False,False,True,False,False,,,,,,,,{'$oid': '5d48eb4a4e77

In [58]:
logs_customer = dict_to_column(['traits'],logs_json)

100%|██████████| 1019669/1019669 [00:00<00:00, 1665728.71it/s]


In [110]:
# 책을 구매한 이력이 있는 사람들의 로그 기록
logs_book_buyer = logs_customer[logs_customer['customerId'].isin(orders_items[orders_items['product_id_$oid']== '60236e092c593a001ace9ffa']['$oid'])]

In [113]:
logs_book_buyer['customerId'][-30:]

1019402    5d968f800dabe405b156d87e
1019408    5f7fdd7348060c52caf165c6
1019419    5d62c2ce4e77525ec5ca15c5
1019424    5f6dc33648060c52caf1578b
1019447    5d62c2ce4e77525ec5ca15c5
1019457    6062f5cf0ccc56001bc7a235
1019461    5e4fac8d4267e105dfff2082
1019467    5f3c916ba0b91001bf48cf7e
1019488    5dc8e34e2bb59605ca3cfb3b
1019489    5dc8e34e2bb59605ca3cfb3b
1019497    5f3c916ba0b91001bf48cf7e
1019499    5d6b08e619efa30eb2913e00
1019501    5d62c2ce4e77525ec5ca15c5
1019508    5f644fdd48060c52caf1408f
1019527    5d62c2ce4e77525ec5ca15c5
1019528    5d62c2ce4e77525ec5ca15c5
1019529    5da6cfbb0dabe405b156eb76
1019531    5e4fac8d4267e105dfff2082
1019535    5dc8e34e2bb59605ca3cfb3b
1019542    5f3c916ba0b91001bf48cf7e
1019560    5d9a88340dabe405b156df5e
1019573    5d62c2ce4e77525ec5ca15c5
1019588    5d6fd2b219efa30eb29141b6
1019590    5ec3d07de510ee503167aaf1
1019604    5da6cfbb0dabe405b156eb76
1019608    5d62c2ce4e77525ec5ca15c5
1019617    5d6b08e619efa30eb2913e00
1019629    5d968f800dabe405b

In [142]:
logs_book_buyer[logs_book_buyer['customerId'] == '5d62c2ce4e77525ec5ca15c5'].tail()

,_metadata,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,properties,category,name,event,unbundled,bundledIds,bundled,_metadata_unbundled,_metadata_bundledIds,_metadata_bundled,customerId,traits_customerId,locale,library,ip,userAgent,campaign,referrer,search,path,title,url,page_referrer,page_search,page_path,page_title,page_url
1019527,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f9b458b6-b92b-4092-8e57-8875bf1451a3,client,{},ajs-next-d529f3b6929b30843e952e7d267876d7,2022-08-24T11:24:40.690Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T11:24:40.530Z,2022-08-24T11:24:40.797Z,2022-08-24 11:24:40.423,identify,104b90e559bb40ad8aab5faa7712721347e3fda9fc2d86...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d62c2ce4e77525ec5ca15c5,5d62c2ce4e77525ec5ca15c5,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.41...",61.253.178.119,Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G99...,None,https://medistream.co.kr/,,/boards/KMDForest,메디스트림 | Medistream,https://medistream.co.kr/boards/KMDForest,https://medistream.co.kr/,,/boards/KMDForest,메디스트림 | Medistream,https://medistream.co.kr/boards/KMDForest
1019528,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f9b458b6-b92b-4092-8e57-8875bf1451a3,client,{},ajs-next-51651a7a992fb0b8d88d1baae9269a8c,2022-08-24T14:58:11.044Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T14:58:10.789Z,2022-08-24T14:58:11.162Z,2022-08-24 14:58:10.671,identify,104b90e559bb40ad8aab5faa7712721347e3fda9fc2d86...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d62c2ce4e77525ec5ca15c5,5d62c2ce4e77525ec5ca15c5,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.41...",61.253.178.119,Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G99...,None,https://medistream.co.kr/,,/boards/AndroMedi,메디스트림 | Medistream,https://medistream.co.kr/boards/AndroMedi,https://medistream.co.kr/,,/boards/AndroMedi,메디스트림 | Medistream,https://medistream.co.kr/boards/AndroMedi
1019573,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f9b458b6-b92b-4092-8e57-8875bf1451a3,client,{},ajs-next-9fcb8f4d7e0d8228ba6713c14a15bac1,2022-08-24T01:19:34.794Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T01:19:34.648Z,2022-08-24T01:19:34.838Z,2022-08-24 01:19:34.604,identify,104b90e559bb40ad8aab5faa7712721347e3fda9fc2d86...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d62c2ce4e77525ec5ca15c5,5d62c2ce4e77525ec5ca15c5,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.41...",61.253.178.119,Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G99...,None,https://medistream.co.kr/,,/boards/Job,메디스트림 | Medistream,https://medistream.co.kr/boards/Job,https://medistream.co.kr/,,/boards/Job,메디스트림 | Medistream,https://medistream.co.kr/boards/Job
1019608,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f9b458b6-b92b-4092-8e57-8875bf1451a3,client,{},ajs-next-34a89a3f7abc1b62a8d750cc09670003,2022-08-24T10:05:38.883Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T10:05:37.939Z,2022-08-24T10:05:38.942Z,2022-08-24 10:05:37.880,identify,104b90e559bb40ad8aab5faa7712721347e3fda9fc2d86...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d62c2ce4e77525ec5ca15c5,5d62c2ce4e77525ec5ca15c5,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.41...",61.253.178.119,Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G99...,None,https://medistream.co.kr/,,/boards/KMDForest,메디스트림 | Medistream,https://medistream.co.kr/boards/KMDForest,https://medistream.co.kr/,,/boards/KMDForest,메디스트림 | Medistream,https://medistream.co.kr/boards/KMDForest
1019642,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",0cc84d7c-3719-4de3-8a90-2606abcd1dad,client,{},ajs-next-23d85d2d71aa1174c1aaf38d09e9df71,2022-08-24T10:41:06.715Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-08-24T10:41:04.266Z,2022-08-24T10:41:06.725Z,2022-08-24 10:41:04.256,identify,104b90e559bb40ad8aab5faa7712721347e3fda9fc2d86...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d62c2ce4e77525ec5ca15c5,5d62c2ce4e77525ec5ca15c5,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.41...",61.2

In [125]:
logs_book_buyer = dict_to_column(['context'],logs_book_buyer)

100%|██████████| 144662/144662 [00:00<00:00, 545384.80it/s]
<ipython-input-56-70f8a44fc872>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[column].apply(lambda x: x.get(key, None) if isinstance(x, dict) else None if x==None


In [130]:
logs_book_buyer = dict_to_column(['page'],logs_book_buyer)

100%|██████████| 144662/144662 [00:00<00:00, 607411.19it/s]


In [143]:
logs_book_buyer[logs_book_buyer['customerId'] == '5d6fd2b219efa30eb29141b6']

,_metadata,anonymousId,channel,integrations,messageId,originalTimestamp,projectId,receivedAt,sentAt,timestamp,type,userId,version,properties,category,name,event,unbundled,bundledIds,bundled,_metadata_unbundled,_metadata_bundledIds,_metadata_bundled,customerId,traits_customerId,locale,library,ip,userAgent,campaign,referrer,search,path,title,url,page_referrer,page_search,page_path,page_title,page_url
223,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f16dd99e-e038-48a4-a945-798bda5ddbae,client,{},ajs-next-aadc4e92fa05217bf2c03e05843f7fc8,2022-05-13T10:25:56.058Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T10:25:54.105Z,2022-05-13T10:25:56.058Z,2022-05-13 10:25:54.105,identify,c3ec32296441b1820979781299656e3fe7a04a7505c4a2...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d6fd2b219efa30eb29141b6,5d6fd2b219efa30eb29141b6,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.36...",223.39.244.67,Mozilla/5.0 (Linux; Android 12; SM-G977N) Appl...,None,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr
410,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f16dd99e-e038-48a4-a945-798bda5ddbae,client,{},ajs-next-e5e92c6e9391d1471b050cbd8d9a7836,2022-05-13T10:29:00.470Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T10:28:58.522Z,2022-05-13T10:29:00.470Z,2022-05-13 10:28:58.522,identify,c3ec32296441b1820979781299656e3fe7a04a7505c4a2...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d6fd2b219efa30eb29141b6,5d6fd2b219efa30eb29141b6,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.36...",223.39.244.67,Mozilla/5.0 (Linux; Android 12; SM-G977N) Appl...,None,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr
1481,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f16dd99e-e038-48a4-a945-798bda5ddbae,client,{},ajs-next-836187f3ece9ddb5c5374c0e5a1de700,2022-05-13T10:42:09.741Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T10:42:07.329Z,2022-05-13T10:42:09.741Z,2022-05-13 10:42:07.329,identify,c3ec32296441b1820979781299656e3fe7a04a7505c4a2...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d6fd2b219efa30eb29141b6,5d6fd2b219efa30eb29141b6,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.36...",58.122.5.19,Mozilla/5.0 (Linux; Android 12; SM-G977N) Appl...,None,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr
2034,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",419f1460-7b08-4e80-972c-c24cfcceb373,client,{},ajs-next-ea203f66167440c7f1b4f46c2d18d096,2022-05-13T00:10:45.383Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T00:10:45.829Z,2022-05-13T00:10:45.383Z,2022-05-13 00:10:45.829,identify,c3ec32296441b1820979781299656e3fe7a04a7505c4a2...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d6fd2b219efa30eb29141b6,5d6fd2b219efa30eb29141b6,ko,"{'name': 'analytics.js', 'version': 'next-1.36...",106.245.192.132,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,None,https://nid.naver.com/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr,https://nid.naver.com/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr
2085,"{'bundled': ['Segment.io'], 'bundledIds': [], ...",f16dd99e-e038-48a4-a945-798bda5ddbae,client,{},ajs-next-a37007430dae321b6510ec5fbe1fef48,2022-05-13T10:42:59.790Z,pAzAoFnKNpLfHxRmZ2oDF4,2022-05-13T10:42:57.362Z,2022-05-13T10:42:59.790Z,2022-05-13 10:42:57.362,identify,c3ec32296441b1820979781299656e3fe7a04a7505c4a2...,2,None,None,None,None,[],[],[Segment.io],[],[],[Segment.io],5d6fd2b219efa30eb29141b6,5d6fd2b219efa30eb29141b6,ko-KR,"{'name': 'analytics.js', 'version': 'next-1.36...",58.122.5.19,Mozilla/5.0 (Linux; Android 12; SM-G977N) Appl...,None,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr,https://medistream.co.kr/,,/,(1) 메디스트림 | Medistream,https://medistream.co.kr
...,...,...,...,...,.